01. Initializations

In [12]:
import pandas as pd
import sys
import json
import os
import datetime


root_folder = '../CREME_dataset/modify'

Test

In [13]:
df = pd.read_csv(os.path.join(root_folder, "original_relabel_syslog.csv"))
# df = pd.read_csv(os.path.join(root_folder, "label_accounting.csv"))
df.info()
df.describe()
# df.head(100)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 235952 entries, 0 to 235951
Data columns (total 11 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   Time             235952 non-null  object 
 1   HostName         235952 non-null  object 
 2   Component        235952 non-null  object 
 3   PID_or_IP        235911 non-null  object 
 4   Content          235952 non-null  object 
 5   EventId          235952 non-null  object 
 6   EventTemplate    235952 non-null  object 
 7   ParameterList    235952 non-null  object 
 8   Timestamp        235952 non-null  float64
 9   Label            235952 non-null  int64  
 10  Label_lifecycle  235952 non-null  int64  
dtypes: float64(1), int64(2), object(8)
memory usage: 19.8+ MB


,Timestamp,Label,Label_lifecycle
count,2.359520e+05,235952.000000,235952.000000
mean,1.678097e+09,2.129128,2.274649
std,2.861132e+03,3.846869,1.856044
min,1.678093e+09,0.000000,0.000000
25%,1.678095e+09,0.000000,0.000000
50%,1.678098e+09,1.000000,2.000000
75%,1.678100e+09,1.000000,4.000000
max,1.678102e+09,17.000000,5.000000


In [14]:
col_list = ['HostName', 'Content', 'Timestamp', 'Label', 'Label_lifecycle', 'PID_or_IP', 'Component']
df = pd.read_csv(os.path.join(root_folder, "original_relabel_syslog.csv"), usecols=col_list)
# df = pd.read_csv(os.path.join(root_folder, "label_accounting.csv"))

df = df.drop(columns=['Component'])
df['count'] = 1
kargs = {
    'Label': pd.NamedAgg(column='Label', aggfunc="first"),
    'Label_lifecycle': pd.NamedAgg(column='Label_lifecycle', aggfunc="first")
}
df = df.sort_values('Timestamp')
# df = df.sort_values('TIMESTAMP')
for column in df:
    # if column not in ('Hostname', 'PID', 'TIMESTAMP', 'Label', 'Label_lifecycle'):
    if column not in ('HostName', 'PID_or_IP', 'Timestamp', 'Label', 'Label_lifecycle'):
        if '_' not in column and column not in ('count', 'Content'):
            kargs[f'{column}_mean'] = pd.NamedAgg(column=column, aggfunc="mean")
            kargs[f'{column}_max'] = pd.NamedAgg(column=column, aggfunc="max")
            kargs[f'{column}_min'] = pd.NamedAgg(column=column, aggfunc="min")
            kargs[f'{column}_std'] = pd.NamedAgg(column=column, aggfunc="std")
        elif column == 'Content':
            kargs[column] = pd.NamedAgg(column=column, aggfunc=lambda x: '. '.join(x))
        else:
            kargs[column] = pd.NamedAgg(column=column, aggfunc="sum")
# df = df.groupby(['HostName', 'PID_or_IP', 'Timestamp', 'Label', 'Label_lifecycle']).sum(numeric_only=True).reset_index()
# df['TIMESTAMP'] //= 10
df['Timestamp'] //= 5
# df = df.groupby(['Hostname', 'PID', 'TIMESTAMP']).agg(**kargs).reset_index()
df = df.groupby(['HostName', 'PID_or_IP', 'Timestamp']).agg(**kargs).reset_index()
df = df.fillna(0)


df.to_csv(os.path.join(root_folder, "aggregated_syslog.csv"), index=False)
# df.to_csv(os.path.join(root_folder, "aggregated_accounting.csv"), index=False)

In [15]:
df = pd.read_csv(os.path.join(root_folder, "aggregated_syslog.csv"))
import random
# df = pd.read_csv(os.path.join(root_folder, "aggregated_accounting.csv"))
df.info()
df.describe()
# df = df[df.Label!=0].reset_index()
for _ in range(100):
    i = random.randint(0, len(df))
    print(df['Label'][i], df['Label_lifecycle'][i], df['Content'][i])

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29012 entries, 0 to 29011
Data columns (total 7 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   HostName         29012 non-null  object 
 1   PID_or_IP        29012 non-null  object 
 2   Timestamp        29012 non-null  float64
 3   Label            29012 non-null  int64  
 4   Label_lifecycle  29012 non-null  int64  
 5   Content          29012 non-null  object 
 6   count            29012 non-null  int64  
dtypes: float64(1), int64(3), object(3)
memory usage: 1.5+ MB
0 0 localhost (192.168.56.151[192.168.56.151]) - FTP session closed.. localhost (192.168.56.151[192.168.56.151]) - USER vulnerable-machine: Login successful.. localhost (192.168.56.151[192.168.56.151]) - FTP session opened.
6 3 localhost (192.168.56.142[192.168.56.142]) - FTP session opened.. localhost (192.168.56.142[192.168.56.142]) - USER non-vulnerable-machine-2: Login successful.. localhost (192.168.56.142[1

In [16]:
import re
from sentence_transformers import SentenceTransformer
from sklearn.decomposition import PCA

df = pd.read_csv(os.path.join(root_folder, "aggregated_syslog.csv"))

content = list(df['Content'])
print(len(content))
for i in range(len(content)):
    content[i] = re.sub('(?:[0-9]{1,3}\.){3}[0-9]{1,3}', '<*>', content[i])
    content[i] = re.sub('port [0-9]{1,5}', 'port <*>', content[i])
corpus = list(set(content))

embedder = SentenceTransformer('all-MiniLM-L6-v2') #(4297, 384)
corpus_embeddings = embedder.encode(corpus, show_progress_bar=True)
print(corpus_embeddings.shape)

pca = PCA(n_components=0.999)
f = pca.fit(corpus_embeddings.T)
corpus_embeddings = f.components_.T
print(corpus_embeddings.shape)

d = dict(zip(corpus, corpus_embeddings))

for i in range(corpus_embeddings.shape[1]):
    df[f'WE_{i}'] = list(map(lambda s: d[s][i], content))

df.to_csv(os.path.join(root_folder, "NLP_aggregated_syslog.csv"), index=False)

29012


Batches:   0%|          | 0/135 [00:00<?, ?it/s]

(4297, 384)
(4297, 80)


In [17]:
df = pd.read_csv(os.path.join(root_folder, "NLP_aggregated_syslog.csv"))
df.info()
df.describe()

/var/folders/hg/c084_sjj0_1dhg3qjvswf_mh0000gn/T/ipykernel_60866/431733217.py:1: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(os.path.join(root_folder, "NLP_aggregated_syslog.csv"))


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29012 entries, 0 to 29011
Data columns (total 87 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   HostName         29012 non-null  object 
 1   PID_or_IP        29012 non-null  object 
 2   Timestamp        29012 non-null  float64
 3   Label            29012 non-null  int64  
 4   Label_lifecycle  29012 non-null  int64  
 5   Content          29012 non-null  object 
 6   count            29012 non-null  int64  
 7   WE_0             29012 non-null  float64
 8   WE_1             29012 non-null  float64
 9   WE_2             29012 non-null  float64
 10  WE_3             29012 non-null  float64
 11  WE_4             29012 non-null  float64
 12  WE_5             29012 non-null  float64
 13  WE_6             29012 non-null  float64
 14  WE_7             29012 non-null  float64
 15  WE_8             29012 non-null  float64
 16  WE_9             29012 non-null  float64
 17  WE_10       

,Timestamp,Label,Label_lifecycle,count,WE_0,WE_1,WE_2,WE_3,WE_4,WE_5,...,WE_70,WE_71,WE_72,WE_73,WE_74,WE_75,WE_76,WE_77,WE_78,WE_79
count,2.901200e+04,29012.000000,29012.000000,29012.000000,29012.000000,29012.000000,29012.000000,29012.000000,29012.000000,29012.000000,...,29012.000000,29012.000000,29012.000000,29012.000000,29012.000000,29012.000000,29012.000000,29012.000000,29012.000000,29012.000000
mean,3.356195e+08,2.539260,1.746691,8.131497,0.005222,0.012434,-0.001506,-0.081959,-0.052425,-0.002080,...,-0.016982,-0.008834,-0.001418,-0.007781,-0.012504,-0.006250,0.004026,-0.007918,-0.002457,0.019065
std,5.832589e+02,4.501086,1.935603,31.954559,0.004691,0.008007,0.007683,0.055308,0.059018,0.020521,...,0.040113,0.032173,0.017547,0.035557,0.028960,0.014841,0.018554,0.030303,0.050439,0.027908
min,3.356186e+08,0.000000,0.000000,1.000000,-0.000989,-0.004429,-0.018178,-0.127633,-0.099458,-0.089944,...,-0.223706,-0.213092,-0.100622,-0.118711,-0.123145,-0.152670,-0.246465,-0.176286,-0.219703,-0.120869
25%,3.356191e+08,0.000000,0.000000,3.000000,0.002947,0.011155,-0.000664,-0.125195,-0.096130,-0.015469,...,-0.037123,-0.031387,-0.012713,-0.034375,-0.017761,-0.014808,-0.007048,-0.027729,-0.054757,0.001509
50%,3.356196e+08,0.000000,0.000000,3.000000,0.003127,0.011554,0.000084,-0.123118,-0.094684,-0.010519,...,-0.006631,-0.002871,-0.004273,-0.005945,-0.010040,-0.006574,0.001898,-0.010694,0.001645,0.023678
75%,3.356200e+08,2.000000,4.000000,5.000000,0.005015,0.012187,0.000493,-0.005410,-0.001886,0.003281,...,0.010523,0.012662,0.004987,0.013807,0.001358,-0.001645,0.013772,0.011221,0.015690,0.036579
max,3.356204e+08,17.000000,5.000000,500.000000,0.018643,0.028636,0.050751,0.011419,0.121082,0.074966,...,0.150457,0.153369,0.150890,0.236901,0.150202,0.199328,0.175722,0.116542,0.158040,0.147807
